### Imports and installs

In [1]:
# !pip3 install dowhy --user

In [2]:
import dowhy
from dowhy import CausalModel

#NOTE: Restart kernel before import
%reload_ext autoreload
%autoreload 2
%reload_ext Cython

import numpy as np
import warnings
import pandas as pd
from pathlib import Path
import time
import decimal
import sys
sys.path.append(str(Path.home()) + '/rpi-canary-eval/modules')

import data_process, data_functions, diabetes_onset, dowhy_methods
from path import path_dict

### Pull Causal Data from S3 - alternatively can generate if any new data introduced

In [3]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/CAUSAL.p ../../temp/CAUSAL.p --quiet

In [4]:
data = pd.read_pickle(path_dict['generated_causal_data_path'])

In [5]:
data.ALLOW_AMT = data.ALLOW_AMT.apply(data_process.log_)
#-- extra
data.ALLOW_IP = data.ALLOW_IP.apply(data_process.log_)
data.ALLOW_ER = data.ALLOW_ER.apply(data_process.log_)

In [6]:
#--extra : no PERS_ID in X
col_seq = ['AGE_AT_MIDMONTH','MBR_GNDR','ALLOW_AMT','ALLOW_ER','ALLOW_IP','ELIXHAUSER','ELX_GRP_1', 'ELX_GRP_2', 'ELX_GRP_3', 'ELX_GRP_4', 'ELX_GRP_5','ELX_GRP_6','ELX_GRP_7','ELX_GRP_8','ELX_GRP_9',
           'ELX_GRP_10','ELX_GRP_11','ELX_GRP_12','ELX_GRP_13','ELX_GRP_14','ELX_GRP_15','ELX_GRP_16','ELX_GRP_18', 'ELX_GRP_19','ELX_GRP_20','ELX_GRP_21','ELX_GRP_22','ELX_GRP_23','ELX_GRP_24',
           'ELX_GRP_25','ELX_GRP_26','ELX_GRP_27','ELX_GRP_28','ELX_GRP_29','ELX_GRP_30','ELX_GRP_31', 'ELX_SCORE']

In [7]:
data.head()

AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP BIOMETRIC_CATEGORY  \
0               73   0.000000       0.0       0.0         ELX_GRP_12   
1               73   5.575949       0.0       0.0         ELX_GRP_12   
2               71   5.846439       0.0       0.0         ELX_GRP_12   
3               69   5.789960       0.0       0.0         ELX_GRP_12   
4               69   5.081404       0.0       0.0         ELX_GRP_12   

   ELIXHAUSER  ELX_GRP_1  ELX_GRP_10  ELX_GRP_11  ELX_GRP_12  ...  ELX_GRP_9  \
0         0.0        0.0         0.0         0.0         0.0  ...        0.0   
1         0.0        0.0         0.0         0.0         0.0  ...        0.0   
2         0.0        0.0         0.0         0.0         0.0  ...        0.0   
3         2.0        0.0         0.0         1.0         0.0  ...        0.0   
4         2.0        0.0         0.0         0.0         0.0  ...        0.0   

   ELX_SCORE  MBR_GNDR  OUTCOME  OUTCOME2  OUTCOME3  PERS_ID  RESULT_VALUE  \
0        0.0         0      0.0       NaN         1  2654945           0.0   
1        0.0         0      0.0       NaN         1  2655294           0.0   
2        0.0         0      0.0       NaN         1  2668219           0.0   
3        0.0         0      0.0       NaN         1  2680996           0.0   
4       -2.0         0      0.0       NaN         1  2690889           0.0   

   RESULT_VALUE2  TREATED  
0              1        1  
1              1        1  
2              1        1  
3              1        1  
4              1        1  

[5 rows x 45 columns]

### Split X-Y-T

In [8]:
# X1 = data[["MBR_GNDR", "AGE_AT_MIDMONTH", "ALLOW_AMT"]]
# X2 = data.loc[:,"CC_ADHD":'CC_VISUAL_IMPAIR']
# X = pd.concat([X1, X2], axis=1)

In [9]:
# X.head()

In [10]:
#-- extra
X = data[col_seq]

In [11]:
Y = data[["OUTCOME"]]
Y2 = data[["OUTCOME3"]]
T = data[["TREATED"]].astype({"TREATED":"bool"}, copy=False)

### >> Treatment effect in original space

In [13]:
treatment_label='TREATED'
outcome_label = 'OUTCOME'
common_causes_list = X.columns.tolist()
target_units = 'att'

warnings.filterwarnings("ignore", category=FutureWarning)
model, identified_estimand, estimate = dowhy_methods.run_do_why(pd.concat([X,Y,T], axis=1), treatment_label, outcome_label, common_causes_list, dowhy_methods.methods['PSS'], target_units)

Step 1: Causal Model Created
Step 2: Estimand Identified


/home/nneehal/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Step 3: Estimated the treatment effect
Causal Estimate is -0.0005060047004540759
Step 4: Refutation


/home/nneehal/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


a. After random common cause refutation --
Refute: Add a Random Common Cause
Estimated effect:(-0.0005060047004540759,)
New effect:(-0.0007735535963592515,)



/home/nneehal/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/nneehal/.local/lib/python3.6/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:92: RuntimeWarning: invalid value encountered in double_scalars
  est = (weighted_outcomes['effect'] * weighted_outcomes[treatment_sum_name]).sum() / total_treatment_population


b. After placebo treatment refutation --
Refute: Use a Placebo Treatment
Estimated effect:(-0.0005060047004540759,)
New effect:(nan,)



/home/nneehal/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


c. After random subset refutation --
Refute: Use a subset of data
Estimated effect:(-0.0005060047004540759,)
New effect:(-0.0009817757869546543,)



### Pull Latent Model from S3

In [ ]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/MODELS/TF_Latent_Model ../../temp/TF_Latent_Model --quiet

In [14]:
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
import tensorflow.keras.backend as K
import keras_model

model = keras.models.load_model("../../temp/TF_Latent_Model_m2", compile=False)

### Data Features Transform from original space to latent space

In [15]:
%%time
latent = model.predict(X)

CPU times: user 29.9 s, sys: 9.66 s, total: 39.5 s
Wall time: 14 s


In [16]:
latent_dim = 8
latent_col_labels = []
for i in range(latent_dim):
    latent_col_labels.append("x"+str(i+1))
latent = pd.DataFrame(latent, columns=latent_col_labels)

In [17]:
new_data = pd.concat([latent, Y, T], axis=1)

### >> Treatment effect in latent space

In [18]:
treatment_label='TREATED'
outcome_label = 'OUTCOME'
common_causes_list = latent.columns.tolist()
target_units = 'att'

warnings.filterwarnings("ignore", category=FutureWarning)
model, identified_estimand, estimate = dowhy_methods.run_do_why(new_data.copy(), treatment_label, outcome_label, common_causes_list, dowhy_methods.methods['PSS'], target_units)

Step 1: Causal Model Created
Step 2: Estimand Identified


/home/nneehal/.local/lib/python3.6/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:92: RuntimeWarning: invalid value encountered in double_scalars
  est = (weighted_outcomes['effect'] * weighted_outcomes[treatment_sum_name]).sum() / total_treatment_population


Step 3: Estimated the treatment effect
Causal Estimate is nan
Step 4: Refutation


/home/nneehal/.local/lib/python3.6/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:92: RuntimeWarning: invalid value encountered in double_scalars
  est = (weighted_outcomes['effect'] * weighted_outcomes[treatment_sum_name]).sum() / total_treatment_population


a. After random common cause refutation --
Refute: Add a Random Common Cause
Estimated effect:(nan,)
New effect:(nan,)



/home/nneehal/.local/lib/python3.6/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:92: RuntimeWarning: invalid value encountered in double_scalars
  est = (weighted_outcomes['effect'] * weighted_outcomes[treatment_sum_name]).sum() / total_treatment_population


b. After placebo treatment refutation --
Refute: Use a Placebo Treatment
Estimated effect:(nan,)
New effect:(nan,)

c. After random subset refutation --
Refute: Use a subset of data
Estimated effect:(nan,)
New effect:(nan,)



/home/nneehal/.local/lib/python3.6/site-packages/dowhy/causal_estimators/propensity_score_stratification_estimator.py:92: RuntimeWarning: invalid value encountered in double_scalars
  est = (weighted_outcomes['effect'] * weighted_outcomes[treatment_sum_name]).sum() / total_treatment_population


### Optional -- latent df to S3

In [19]:
new_data = new_data.loc[:,"x1":"TREATED"]

In [20]:
new_data.to_pickle("./CAUSAL_LATENT.p")

In [21]:
!pwd

/mnt/home/nneehal/rpi-canary-eval/notebooks/experiments


In [22]:
#save in S3
import boto3
from pathlib import Path
s3 = boto3.client('s3')
bucket = 'cdphp-s3-us-e-p-pond'
s3.upload_file( str(Path.home()) + '/rpi-canary-eval/notebooks/experiments/CAUSAL_LATENT.p',
                bucket, 
                'rpi/nneehal/DATA/CAUSAL_LATENT.p',
                ExtraArgs = {'ACL':"bucket-owner-full-control", 'ServerSideEncryption':'AES256'})